In [1]:
!pip install scikit-learn-intelex
!pip install numba
!pip install gradio
from sklearnex import patch_sklearn, config_context
patch_sklearn()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


##Intel oneAPI Shortest Path Algorithm

In [ ]:
import pandas as pd
import itertools
import numba
import numpy as np
from sklearnex import patch_sklearn

# Patch scikit-learn to optimize with Intel's extension
patch_sklearn()

# Load the dataset
file_path = 'indian-cities-dataset.csv'
data = pd.read_csv(file_path)

# Create a graph as a dictionary (not using numba here)
def create_graph(data):
    graph = {}
    for index, row in data.iterrows():
        if row['Origin'] not in graph:
            graph[row['Origin']] = []
        graph[row['Origin']].append((row['Destination'], row['Distance']))

        if row['Destination'] not in graph:
            graph[row['Destination']] = []
        graph[row['Destination']].append((row['Origin'], row['Distance']))
    return graph

# Function to convert the graph to a NumPy-friendly format for distance calculation
def convert_graph_to_matrix(graph, cities):
    n = len(cities)
    city_index = {city: idx for idx, city in enumerate(cities)}  # Map city names to indices
    distance_matrix = np.full((n, n), np.inf)  # Initialize with infinity for non-edges

    for city, neighbors in graph.items():
        if city in cities:  # Ensure city is in the list of cities we care about
            for neighbor, distance in neighbors:
                if neighbor in cities:  # Only consider neighbors that are in the list
                    i, j = city_index[city], city_index[neighbor]
                    distance_matrix[i][j] = distance

    return distance_matrix, city_index

# Optimized function to get the distance of a path using numba
@numba.njit
def get_path_distance(distance_matrix, path_indices):
    total_distance = 0
    for i in range(len(path_indices) - 1):
        distance = distance_matrix[path_indices[i], path_indices[i + 1]]
        if distance == np.inf:
            return np.inf  # Return inf if the path is not valid
        total_distance += distance
    return total_distance

# Finding all possible paths and the optimal one
def find_optimal_path(start_city, cities_to_cover):
    graph = create_graph(data)
    all_cities = [start_city] + cities_to_cover

    # Convert graph to distance matrix
    distance_matrix, city_index = convert_graph_to_matrix(graph, all_cities)

    # Generate all permutations of the cities to cover
    all_possible_paths = []
    for perm in itertools.permutations(cities_to_cover):
        path = [start_city] + list(perm)
        all_possible_paths.append(path)

    # Calculate distances for each path
    all_paths_distances = np.empty(len(all_possible_paths))

    for idx, path in enumerate(all_possible_paths):
        # Convert path to indices for distance matrix
        path_indices = np.array([city_index[city] for city in path])
        all_paths_distances[idx] = get_path_distance(distance_matrix, path_indices)

    # Find the optimal path if there are valid paths
    if np.isfinite(all_paths_distances).any():  # If there's at least one valid path
        optimal_idx = np.argmin(all_paths_distances)
        optimal_path = all_possible_paths[optimal_idx]
        optimal_distance = all_paths_distances[optimal_idx]
    else:
        optimal_path = None
        optimal_distance = np.inf

    return optimal_path, optimal_distance, all_possible_paths, all_paths_distances

# Example usage
start_city = "Agra"
cities_to_cover = ["Delhi", "Lucknow", "Kanpur"]
print("Origin:", start_city)
print("Cities to cover:", cities_to_cover)
optimal_path, optimal_distance, all_possible_paths, path_distances = find_optimal_path(start_city, cities_to_cover)
print("Optimal Path: ", optimal_path)
print("Optimal Distance: ", optimal_distance)


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Origin: Agra
Cities to cover: ['Delhi', 'Lucknow', 'Kanpur']
Optimal Path:  ['Agra', 'Delhi', 'Lucknow', 'Kanpur']
Optimal Distance:  885.0


In [8]:
import pandas as pd
import itertools
import numba
import numpy as np
from sklearnex import patch_sklearn
import gradio as gr

# Patch scikit-learn to optimize with Intel's extension
patch_sklearn()

# Load the dataset
file_path = 'indian-cities-dataset.csv'
data = pd.read_csv(file_path)

# Create a graph as a dictionary (not using numba here)
def create_graph(data):
    graph = {}
    for index, row in data.iterrows():
        if row['Origin'] not in graph:
            graph[row['Origin']] = []
        graph[row['Origin']].append((row['Destination'], row['Distance']))

        if row['Destination'] not in graph:
            graph[row['Destination']] = []
        graph[row['Destination']].append((row['Origin'], row['Distance']))
    return graph

# Function to convert the graph to a NumPy-friendly format for distance calculation
def convert_graph_to_matrix(graph, cities):
    n = len(cities)
    city_index = {city: idx for idx, city in enumerate(cities)}  # Map city names to indices
    distance_matrix = np.full((n, n), np.inf)  # Initialize with infinity for non-edges

    for city, neighbors in graph.items():
        if city in cities:  # Ensure city is in the list of cities we care about
            for neighbor, distance in neighbors:
                if neighbor in cities:  # Only consider neighbors that are in the list
                    i, j = city_index[city], city_index[neighbor]
                    distance_matrix[i][j] = distance

    return distance_matrix, city_index

# Optimized function to get the distance of a path using numba
@numba.njit
def get_path_distance(distance_matrix, path_indices):
    total_distance = 0
    for i in range(len(path_indices) - 1):
        distance = distance_matrix[path_indices[i], path_indices[i + 1]]
        if distance == np.inf:
            return np.inf  # Return inf if the path is not valid
        total_distance += distance
    return total_distance

# Finding all possible paths and the optimal one
def find_optimal_path(start_city, cities_to_cover):
    graph = create_graph(data)
    all_cities = [start_city] + cities_to_cover

    # Convert graph to distance matrix
    distance_matrix, city_index = convert_graph_to_matrix(graph, all_cities)

    # Generate all permutations of the cities to cover
    all_possible_paths = []
    for perm in itertools.permutations(cities_to_cover):
        path = [start_city] + list(perm)
        all_possible_paths.append(path)

    # Calculate distances for each path
    all_paths_distances = np.empty(len(all_possible_paths))

    for idx, path in enumerate(all_possible_paths):
        # Convert path to indices for distance matrix
        path_indices = np.array([city_index[city] for city in path])
        all_paths_distances[idx] = get_path_distance(distance_matrix, path_indices)

    # Find the optimal path if there are valid paths
    if np.isfinite(all_paths_distances).any():  # If there's at least one valid path
        optimal_idx = np.argmin(all_paths_distances)
        optimal_path = all_possible_paths[optimal_idx]
        optimal_distance = all_paths_distances[optimal_idx]
    else:
        optimal_path = None
        optimal_distance = np.inf

    return optimal_path, optimal_distance

# Gradio function to process the input and display the result
def optimize_route(start_city, cities_to_cover):
    cities_to_cover = [city.strip() for city in cities_to_cover.split(',')]  # Parse input
    optimal_path, optimal_distance = find_optimal_path(start_city, cities_to_cover)
    if optimal_path is None:
        return "No valid path found.", "N/A"
    else:
        return f" -> ".join(optimal_path), f"{optimal_distance} km"

# Create Gradio interface using updated components
interface = gr.Interface(
    fn=optimize_route,
    inputs=[
        gr.Textbox(label="Start City"),
        gr.Textbox(label="Cities to Cover (comma-separated)")
    ],
    outputs=[
        gr.Textbox(label="Optimal Path"),
        gr.Textbox(label="Optimal Distance (km)")
    ],
    title="Supply Chain Route Optimizer",
    description="Enter a start city and a list of cities to cover, and get the optimal path and distance."
)

# Launch the Gradio app
interface.launch()


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://31948761077422ca5d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
